In [2]:
# libraries
from dotenv import load_dotenv

# load environment variables from .env file
_ = load_dotenv("../../.env")

In [31]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
import time


In [32]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [33]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    time.sleep(1)
    print("N1")
    return {
        # "lnode": "node_1",
        "count": 1,
    }


def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    time.sleep(2)
    print("N2")
    return {
        # "lnode": "node_2",
        "count": 1,
    }


def node3(state: AgentState):
    print(f"node3, count:{state['count']}")
    for i in range(3):
        time.sleep(1)
        print("N3",i)
    return {
        # "lnode": "node_3",
        "count": 1,
    }


def node4(state: AgentState):
    print(f"node4, count:{state['count']}")
    for i in range(3):
        time.sleep(1.5)
        print("N4",i)
    return {
        # "lnode": "node_4",
        "count": 1,
    }


def node5(state: AgentState):
    print(f"node5, count:{state['count']}")
    for i in range(3):
        time.sleep(2)
        print("N5",i)
    return {
        # "lnode": "node_5",
        "count": 1,
    }


def node6(state: AgentState):
    print(f"node6, count:{state['count']}")
    for i in range(6):
        time.sleep(1)
        print("N6",i)
    return {
        # "lnode": "node_6",
        "count": 1,
    }


def should_continue(state):
    return state["count"] < 3

builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)
builder.add_node("Node3", node3)
builder.add_node("Node4", node4)
builder.add_node("Node5", node5)
builder.add_node("Node6", node6)

builder.add_edge("Node1", "Node2")
builder.add_edge("Node2", "Node3")
builder.add_edge("Node2", "Node4")
builder.add_edge("Node2", "Node5")
builder.add_edge("Node3", "Node6")
builder.add_edge("Node4", "Node6")
builder.add_edge("Node5", "Node6")
builder.add_edge("Node6", END)

# builder.add_conditional_edges("Node2", should_continue, {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [34]:
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(checkpointer=checkpointer)
# graph = builder.compile()

In [35]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread)

node1, count:0
N1
node2, count:1
N2
node3, count:2
node4, count:2
node5, count:2
N3 0
N4 0
N3 1
N5 0
N3 2
N4 1
N5 1
N4 2
N5 2
node6, count:5
N6 0
N6 1
N6 2
N6 3
N6 4
N6 5


{'scratch': 'hi', 'count': 6}

In [36]:
graph.get_state(thread)

StateSnapshot(values={'scratch': 'hi', 'count': 6}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-d3fb-67c6-8004-10871c623020'}}, metadata={'source': 'loop', 'writes': {'Node6': {'count': 1}}, 'step': 4}, created_at='2025-02-24T09:45:16.313594+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-9ab7-6319-8003-541b6828f99f'}})

In [37]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'scratch': 'hi', 'count': 6}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-d3fb-67c6-8004-10871c623020'}}, metadata={'source': 'loop', 'writes': {'Node6': {'count': 1}}, 'step': 4}, created_at='2025-02-24T09:45:16.313594+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-9ab7-6319-8003-541b6828f99f'}}) 

StateSnapshot(values={'scratch': 'hi', 'count': 5}, next=('Node6',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-9ab7-6319-8003-541b6828f99f'}}, metadata={'source': 'loop', 'writes': {'Node3': {'count': 1}, 'Node4': {'count': 1}, 'Node5': {'count': 1}}, 'step': 3}, created_at='2025-02-24T09:45:10.308738+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-6172-64b7-8002-a699ac76736e'}}) 

StateSnapshot(values={'scratch': 'hi', 'count': 2}, next=('Node3', 'No

In [38]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-d3fb-67c6-8004-10871c623020'}} 6
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-9ab7-6319-8003-541b6828f99f'}} 5
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-6172-64b7-8002-a699ac76736e'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-4e5a-68d1-8001-af1e449315d5'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-44cb-64e1-8000-6b123fccf3d1'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-44c6-61c6-bfff-7c6015b52caf'}} 0


In [39]:
states[-3]

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1eff2940-4e5a-68d1-8001-af1e449315d5'}}

In [40]:
graph.get_state(states[-3])

StateSnapshot(values={'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-4e5a-68d1-8001-af1e449315d5'}}, metadata={'source': 'loop', 'writes': {'Node1': {'count': 1}}, 'step': 1}, created_at='2025-02-24T09:45:02.301614+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-44cb-64e1-8000-6b123fccf3d1'}})

In [41]:
graph.invoke(None, states[-3])

node3, count:2
node4, count:2
node5, count:2
N3 0
N4 0
N3 1
N5 0
N3 2
N4 1
N5 1
N4 2
N5 2
node6, count:5
N6 0
N6 1
N6 2
N6 3
N6 4
N6 5


{'scratch': 'hi', 'count': 6}

In [42]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-a7a9-6bcc-8004-f8551e30259f'}} 6
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-6e6a-67b4-8003-f4e32f0cade2'}} 5
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-351f-644d-8002-4b5e040a06ca'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-d3fb-67c6-8004-10871c623020'}} 6
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-9ab7-6319-8003-541b6828f99f'}} 5
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-6172-64b7-8002-a699ac76736e'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-4e5a-68d1-8001-af1e449315d5'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2940-44cb-64e1-8000-6b123fccf3d1'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkp

In [43]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'scratch': 'hi', 'count': 6}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-a7a9-6bcc-8004-f8551e30259f'}}, metadata={'source': 'loop', 'writes': {'Node6': {'count': 1}}, 'step': 4}, created_at='2025-02-24T09:46:05.353466+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-6e6a-67b4-8003-f4e32f0cade2'}}) 

StateSnapshot(values={'scratch': 'hi', 'count': 5}, next=('Node6',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-6e6a-67b4-8003-f4e32f0cade2'}}, metadata={'source': 'loop', 'writes': {'Node3': {'count': 1}, 'Node4': {'count': 1}, 'Node5': {'count': 1}}, 'step': 3}, created_at='2025-02-24T09:45:59.350492+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1eff2942-351f-644d-8002-4b5e040a06ca'}}) 

StateSnapshot(values={'scratch': 'hi', 'count': 2}, next=('Node3', 'No